In [1]:
from lxml import etree

In [2]:
from typing import List, Tuple

In [3]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = str(column.text).strip()
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels

In [8]:
texts, labels = load_sentirueval_2016('bank_train_2016.xml')

In [9]:
print('Number of texts is {0}, number of labels is {1}.'.format(len(texts), len(labels)))

Number of texts is 9392, number of labels is 9392.


In [10]:
import random

In [11]:
for idx in random.choices(list(range(len(texts))), k=20):
    print('{0} => {1}'.format(labels[idx], texts[idx]))

neutral => Мошенники совсем обленились - уже даже не пытаются подделать сервисное сообщение от Сбербанка, что "ваша карта заблокирована")
neutral => RT @CrokeAdiella: у альфабанка картинки для рекламы с шаттерстока, о оказывается
negative => С этим "Альфа Банком" столько недостоверной инфы было распространено в СМИ в свое время. Взять тот же "Полный истец" 2004 года
neutral => Займ до зарплаты в Калачинске | Работа, вакансии в Калачинске — Яндекс Работа | Банк Москвы: Работа в Калачинс... http://t.co/7ArMjM6bjA
neutral => документ подписали президент, председатель правления сбербанка россии герман греф и президент республики татарстан рустам минниханов
positive => RT @enobrghoml: Россельхозбанк планирует расширить кредитную поддержку АПК.
neutral => инструкция клиент сбербанк установке http://t.co/SRectUZmjz по
neutral => перекредитование потребительских кредитов в сбербанке http://t.co/Ws9u2lcWs0
neutral => Заявление на погашение кредита в газпромбанк через интернет http://t.co/meMpur

In [12]:
positive_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'positive']
negative_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'negative']

In [13]:
for cur in positive_tweets[:5]: print(cur)

@sawik_shuster @YevhenS Главное чтоб банки СБЕР и ВТБ!!!
Самый выгодный автокредит в втб 24 http://t.co/CDJ5P40x7g
http://t.co/h6r6GdBe4H Легко можно получить денежный кредит ы втб 24 банке
Снижение процентной ставки по кредиту на недвижимость сбербанк http://t.co/MredO898wK
В наш сбербанк пришел американец и попросил сфоткаться с ним у банкомата ;D 
я ваще была в шоке и на фотке полюбому получилась смешная xD


In [14]:
for cur in negative_tweets[:5]: print(cur)

@ShtirliZ_ @Zhukova_olga @winzard @tereshenkov обязательно про сбербанк напишите! Временами похлеще интернет магазинов финты выкручивает)
@Umka_75 @Alfarius2012 @igor_sechin втб и сбер точно. вопрос-блокировка операций или запрет на кредиты?
Канада ввела санкции против Газпромбанка, ВЭБа, Новатэка и российских оборонных компаний
RT @letokot: Если Сбер и ВТБ попали под санкции, будет жопа
RT @360_tv_ru: #Канада ввела санкции против 10 оборонных и сырьевых компаний и банков #РФ. Среди них, в частности, "#Газпромбанк", "#Внешэк…


In [15]:
from nltk import word_tokenize
#import nltk
#nltk.download()

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
vectorizer = CountVectorizer(lowercase=True, tokenizer=word_tokenize)

In [18]:
vectorizer.fit(texts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x000000F1BE7ED6A8>,
                vocabulary=None)

In [19]:
print(vectorizer.get_feature_names()[0:20])

['!', '#', '$', '%', '&', "''", '(', ')', '*^____^*', '*бесится*', '*трипл-смайлик-крик-мунка*', '+', '+3,49', '+300', '+5тыс', '+74955102933', '+в', ',', '-', '--']


In [20]:
print(len(vectorizer.get_feature_names()))

20240


In [21]:
X = vectorizer.transform(texts)

In [22]:
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>


In [23]:
print(texts[0])

http://t.co/YEVHuvVGA1 Взять кредит тюмень альфа банк


In [24]:
print(X[0])

  (0, 6321)	1
  (0, 7199)	1
  (0, 7775)	1
  (0, 9093)	1
  (0, 9295)	1
  (0, 9866)	1
  (0, 12802)	1
  (0, 19056)	1


In [25]:
print(vectorizer.get_feature_names()[6321])

//t.co/yevhuvvga1


In [26]:
print(vectorizer.get_feature_names()[9866])

взять


In [27]:
print(vectorizer.get_feature_names()[19056])

тюмень


In [28]:
from sklearn.feature_extraction.text import TfidfTransformer

In [29]:
transformer = TfidfTransformer().fit(X)

In [30]:
X_transformed = transformer.transform(X)

In [31]:
print(X_transformed[0])

  (0, 19056)	0.5196718117883131
  (0, 12802)	0.1747696254947399
  (0, 9866)	0.2815105414658593
  (0, 9295)	0.20532647450968283
  (0, 9093)	0.26133302396332425
  (0, 7775)	0.10105244280985996
  (0, 7199)	0.09241698661915439
  (0, 6321)	0.7006726731639271


In [32]:
print(vectorizer.get_feature_names()[19056])

тюмень


In [33]:
print(vectorizer.get_feature_names()[7199])

:


In [34]:
tokens_with_IDF = list(zip(vectorizer.get_feature_names(), transformer.idf_))

In [35]:
for feature, idf in tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.417620 => !
3.144655 => #
6.276519 => $
5.636861 => %
5.647910 => &
4.134005 => ''
4.805386 => (
4.443938 => )
9.454573 => *^____^*
9.454573 => *бесится*
9.454573 => *трипл-смайлик-крик-мунка*
9.049108 => +
9.454573 => +3,49
9.454573 => +300
9.454573 => +5тыс
9.454573 => +74955102933
9.454573 => +в
2.753227 => ,
4.420872 => -
8.761426 => --


In [36]:
sorted_tokens_with_IDF = sorted(tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [37]:
for feature, idf in sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => *^____^*
9.454573 => *бесится*
9.454573 => *трипл-смайлик-крик-мунка*
9.454573 => +3,49
9.454573 => +300
9.454573 => +5тыс
9.454573 => +74955102933
9.454573 => +в
9.454573 => -4.8
9.454573 => -5
9.454573 => -6.4
9.454573 => -а
9.454573 => -круто
9.454573 => -осуществление
9.454573 => -подключение
9.454573 => -привлечение
9.454573 => -разменяйте
9.454573 => -россельхозбанка
9.454573 => -сами
9.454573 => -сервис


In [38]:
from sklearn.feature_selection import SelectPercentile, chi2

In [39]:
selector = SelectPercentile(chi2, percentile=20)

In [40]:
selector.fit(X_transformed, labels)

SelectPercentile(percentile=20,
                 score_func=<function chi2 at 0x000000F1BFC44EA0>)

In [41]:
selected_tokens_with_IDF = [tokens_with_IDF[idx] for idx in selector.get_support(indices=True)]

In [42]:
print(len(selected_tokens_with_IDF))

4048


In [43]:
for feature, idf in selected_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.417620 => !
3.144655 => #
5.636861 => %
4.134005 => ''
4.805386 => (
4.443938 => )
9.454573 => *^____^*
9.049108 => +
9.454573 => +5тыс
2.753227 => ,
4.420872 => -
9.454573 => -россельхозбанка
3.175114 => .
7.012226 => ..
3.784692 => ...
9.454573 => ._
6.070183 => /
9.454573 => //t.co/01ftjc0bur
9.454573 => //t.co/0gfmj3s0gi
9.454573 => //t.co/0i4pf81ksd


In [44]:
selected_and_sorted_tokens_with_IDF = sorted(selected_tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [45]:
for feature, idf in selected_and_sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => *^____^*
9.454573 => +5тыс
9.454573 => -россельхозбанка
9.454573 => ._
9.454573 => //t.co/01ftjc0bur
9.454573 => //t.co/0gfmj3s0gi
9.454573 => //t.co/0i4pf81ksd
9.454573 => //t.co/0jzv0jq959
9.454573 => //t.co/0lt5jdmk9l
9.454573 => //t.co/0s03a0v7po
9.454573 => //t.co/0ttwpnjnef
9.454573 => //t.co/0vpcvumoe8
9.454573 => //t.co/0wziwezr91
9.454573 => //t.co/0y05qjok2q
9.454573 => //t.co/1aa3jpbxqh
9.454573 => //t.co/1cjshtzrda
9.454573 => //t.co/1d3bb71qbt
9.454573 => //t.co/1eodyxqq0a
9.454573 => //t.co/1fxzerzsps
9.454573 => //t.co/1kiwdyic6t


In [46]:
from nltk.tokenize import TweetTokenizer
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language='ru')
tokenizer = TweetTokenizer()

def drop_junk(s):
return ' '.join([word for word in tokenizer.tokenize(s) if word.isalpha() or '!' in word or '?' in word \
or '(' in word or ')' in word])

def lemmatization(s):
toks = tokenizer.tokenize(s)
forms = predictor.predict(toks)
return ' '.join([f.normal_form for f in forms])

texts, labels = load_sentirueval_2016('bank_train_2016.xml')

texts = list(map(drop_junk, texts))
texts = list(map(lemmatization, texts))

IndentationError: expected an indented block (<ipython-input-46-f2cba9e16b2c>, line 8)

In [1]:
from stop_words import get_stop_words

stop_words = get_stop_words('ru')
у CountVectorizer есть парамерт stop_words

SyntaxError: invalid syntax (<ipython-input-1-bb09fef1c5b9>, line 4)